In [ ]:
using ValueShapes
using ArraysOfArrays
using StatsBase 
using LinearAlgebra
using Statistics
using Distributions 
using IntervalSets

using HCubature
using JLD
using CPUTime

using Plots
using Colors
using ColorSchemes
using LaTeXStrings

using SpecialFunctions

pyplot(size=(750,500))
line_colors = ColorSchemes.tab20b;

In [ ]:
import PyPlot
plt = PyPlot

SMALL_SIZE = 12
MEDIUM_SIZE = 13
BIGGER_SIZE = 13

plt.rc("font", size=SMALL_SIZE)          # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)    # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
# using Revise 
using BAT

# Density Function: 

Uncomment a density function that needs to be used 

### - Normal Distribution IID:

In [ ]:
# N = 5

# prior_iid = NamedTupleDist(a = [[Normal() for i in 1:N]...],)

# LogTrueIntegral(N) = 0.0 

### - Cauchy Distribution MCMC: 

In [ ]:
# # # Multimodal Caushy Distribution: 

# true_param =(μ1=1, μ2=-1, σ=0.2)
# min_v = -8
# max_v = 8
# N = 15
# lgV = N*log(max_v-min_v); 


# function fun(x; true_param=true_param)
#     tmp = 1
#     for i in eachindex(x)
#         if i > 2
#             tmp *= pdf(Cauchy(true_param.μ1 + true_param.μ2, true_param.σ), x[i])
#         else 
#             tmp *= 0.5*pdf(Cauchy(true_param.μ1, true_param.σ), x[i]) + 0.5*pdf(Cauchy(true_param.μ2, true_param.σ), x[i])
#         end
#     end
#     return tmp
# end

# function LogTrueIntegral(N; max = max_v, min=min_v,  true_param=true_param) 
#     tmp = 0
#     for i in 1:N
#         if i > 2
#             tmp += log(cdf(Cauchy(true_param.μ1 + true_param.μ2,true_param.σ), max_v) - cdf(Cauchy(true_param.μ1 + true_param.μ2,true_param.σ), min_v))
#         else 
#             tmp += log(cdf(Cauchy(true_param.μ1,true_param.σ), max_v) - cdf(Cauchy(true_param.μ1 ,true_param.σ), min_v))
#         end
#     end
#     return tmp
# end

### - Cauchy IID:

In [ ]:
# N = 3

# min_v = -8
# max_v = 8

# prior_iid = NamedTupleDist(a = [[Cauchy(0.0, 0.2) for i in 1:N]...],)

# LogTrueIntegral(N) = 0.0 

### - Gaussian Shell MCMC:

In [ ]:
# # Gaussian Shell Den

N = 2
min_v = -25
max_v = 25

lgV = N*log(max_v-min_v); 

true_param =(λ=zeros(N), r=5, σ=2)

function fun(x; true_param=true_param)
    abs_dist = sqrt(sum((true_param.λ .- x).^2))
    return exp(-(abs_dist - true_param.r)^2/(2*true_param.σ^2)) / sqrt(2 * pi * true_param.σ^2) 
end

function LogTrueIntegral(N; true_param=true_param) 
    g(x; true_param=true_param, N=N) = x[1]^(N-1) * exp( -(x[1]-true_param.r)^2/(2*true_param.σ^2) )
    return log((sqrt(2)*pi^((N-1)/2)) / (gamma(N/2)*true_param.σ)*hcubature(g, [0], [30])[1])
end

# LogTrueIntegral(N)


### - Funel Density MCMC: 

In [ ]:
# N = 15
# min_v = -50
# max_v = 50

# lgV = N*log(max_v-min_v); 

# density_type = Normal(0.0, 1.0)

# function fun(x::Array{Float64}; density_type=density_type)
#     b=0.5
#     μ = 0.0
#     σ = 1.0
#     return pdf(Normal(μ, σ), x[1])*prod(pdf.(Normal(μ, exp(2*b*x[1])), x[2:end]))   
# end

# LogTrueIntegral(N) = 0.0 

# Gerenrate Samples: 

Generate samples according to the given function 

In [ ]:
algorithm = MetropolisHastings()

tuning = AdaptiveMetropolisTuning(
    λ = 0.5,
    α = 0.15..0.35,
    β = 1.5,
    c = 1e-4..1e2
)

convergence = BrooksGelmanConvergence(
    threshold = 1.1,
    corrected = false
)

init = MCMCInitStrategy(
    init_tries_per_chain = 8..128,
    max_nsamples_init = 250,
    max_nsteps_init = 250,
    max_time_init = 180
)

burnin = MCMCBurninStrategy(
    max_nsamples_per_cycle = 1000,
    max_nsteps_per_cycle = 10000,
    max_time_per_cycle = 250,
    max_ncycles = 200
)

HMI_Manual_Settings = BAT.HMISettings(BAT.cholesky_partial_whitening!, 
        1000, 
        1.5, 
        0.1, 
        true, 
        16, 
        true, 
        Dict("cov. weighted result" => BAT.hm_combineresults_covweighted!)
    )

log_likelihood = params -> LogDVal((log(fun(params.a))))

prior = NamedTupleDist(a = [[min_v .. max_v for i in 1:N]...],)

posterior = PosteriorDensity(log_likelihood, prior);

In [ ]:
nchains_ = 10
nsamples_ = 1*10^5
max_time = 150
max_nsteps = 10 * nsamples_;

In [ ]:
# IID sampling: 

# samples = bat_sample(prior_iid, nchains_*nsamples_).result; 

In [ ]:
# MCMC

@time samples, chains = bat_sample(
    posterior, (nsamples_, nchains_), algorithm,
    max_nsteps = max_nsteps,
    max_time = max_time,
    tuning = tuning,
    init = init,
    burnin = burnin,
    convergence = convergence,
    strict = false,
    filter = true
);

In [ ]:
plot(samples, vsel=[1,2,3,4 ], colorbar=false, grid=false, bins=500, size=(1000, 1000))

In [ ]:
hmi_data = BAT.HMIData(unshaped.(samples))

@time BAT.hm_integrate!(hmi_data, settings = HMI_Manual_Settings)

In [ ]:
hmi_data.integralestimates["cov. weighted result"].final.estimate * exp(lgV)

In [ ]:
exp(LogTrueIntegral(N))

## Analyze Data: 

This part plots benchmarking data that has been generated by "run_*.jl" code

In [ ]:
# MPP Path: 

# PATH = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/NormalDistributionData/normal_dist_3-iid.jld"
# PATH = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/NormalDistributionData/normal_dist-ffcor-3.jld"
# TITLE = "AHMI Tests (Multivariate Normal Distribution)"

# PATH = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/CaushyDistributionData/caushy_dist_3.jld"
# PATH = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/CaushyDistributionData/mmod-caushy_dist_4.jld"
# PATH = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/CaushyDistributionData/caushy_dist_3-iid.jld"
# PATH = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/CaushyDistributionData/mmod-caushy_dist_4-ffcor-3.jld"
# TITLE = "AHMI Tests (Multivariate Caushy Distribution)"

# PATH = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/GaussShellDistributionData/gaus_shell_dist_5.jld"
PATH = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/GaussShellDistributionData/gaus_shell_dist_5-ffcor-4.jld"
TITLE = "AHMI Tests (Multivariate Gaussian Shell)"

# PATH = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/FunnelDistributionData/funnel_dist_2.jld"
# PATH = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/FunnelDistributionData/funnel_dist_2-ffcor-4.jld"
# TITLE = "AHMI Tests (Funnel Distribution)"


In [ ]:
saved = load(PATH);

In [ ]:
integrals_ahmi = saved["integrals_ahmi_array"]
integrals_true = saved["integrals_true_array"]
uns_ahmi = saved["uns_ahmi_array"]
dim_array = saved["dim_array"]
sample_size = saved["sample_size"]
ahmi_time_array = saved["ahmi_time_array"]
mcmc_time_array = saved["mcmc_time_array"]

tot_volumes_accepted1_array = saved["tot_volumes_accepted1_array"]
tot_volumes_accepted2_array = saved["tot_volumes_accepted2_array"]

int_estimates_1_array = saved["int_estimates_1_array"]
int_estimates_2_array = saved["int_estimates_2_array"]

tot_volumes_rejected1_array = saved["tot_volumes_rejected1_array"]
tot_volumes_rejected2_array = saved["tot_volumes_rejected2_array"];

dim_range = sort(unique(dim_array));

In [ ]:
integrals = exp.(integrals_ahmi .- integrals_true)
unsertanity_ahmi = exp.(uns_ahmi .- integrals_true);
unsertanity_scatter = std(integrals, dims=1)'

mean_integrals = mean(integrals, dims=1)'
unsertanity_ahmi = mean(unsertanity_ahmi, dims=1)';

plot(dim_range, mean_integrals, ribbon=(unsertanity_scatter, unsertanity_scatter),
    linecolor=line_colors[1], fillcolor=:darkgray, fillalpha=0.3, lw=0, label="Sample std.")

plot!(dim_range, mean_integrals, ribbon=(unsertanity_ahmi, unsertanity_ahmi),
    linecolor=line_colors[1], fillcolor=line_colors[4], fillalpha=0.5, label="Mean AHMI \$\\sigma\$")

hline!([1], 
    linecolor=:gray, 
    lw=3, 
    linealpha=0.7, label="True Integral")

scatter!([dim_array...], [integrals...], #yerror = unsertanity, linealpha=0.1,
    lw=0.0, markeralpha=0.6,
    markercolor=line_colors[3], 
    markerstrokewidth=0.1, label="AHMI results")

plot!(box=true, 
    grid=false, 
    size=(800, 500),
    xticks = dim_range,
#     xlim=(0, 22),
    title = TITLE,
    xlabel = L"N_{dim}", 
    ylabel = L"I/I_{exact}", 
)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(13, 6))

ax.plot(dim_range, mean_integrals[:,1], label=L"$\langle\hat{I}\rangle$")
ax.axhline(1, c="red", label="Truth")

ax.fill_between(dim_range, mean_integrals[:,1] .- unsertanity_scatter[:,1], mean_integrals[:,1] .+ unsertanity_scatter[:,1], alpha=1, color=plt.cm.Blues(0.3), label=L"$\sigma(\hat{I})$" )

ax.plot(dim_range, mean_integrals[:,1] .- unsertanity_ahmi[:,1], alpha=1, ls="--", c="black",  label=L"$\langle\sigma\rangle$",)
ax.plot(dim_range, mean_integrals[:,1] .+ unsertanity_ahmi[:,1], alpha=1, ls="--", c="black")

ax.set_xlabel("# Dimensions")
ax.set_ylabel("Ratio to Truth")

# ins = ax.inset_axes([0.2,0.4,0.3,0.4])
# ins.hist(integrals[:, 1], bins=5, color=plt.cm.Blues(0.3))
# ins.axvline(1, c="red")
# ins.set_xlabel(L"Integral Estimate $\hat{I}$")
# ins.get_yaxis().set_visible(false)
# ins.set_title("# Dimensions = 1")

# ax.set_ylim(0.9, 2)
# ax.set_xlim(2, 18)

ax.legend(loc="upper left", frameon=true, framealpha=0.8, ncol=1)

# fig.savefig("/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/CaushyDistributionData/benchmarking.pdf", bbox_inches = "tight")

In [ ]:
tot_volumes_accepted1_array_mean = mean(tot_volumes_accepted1_array, dims=1)'
tot_volumes_accepted2_array_mean = mean(tot_volumes_accepted2_array, dims=1)'

tot_volumes_accepted1_array_std = std(tot_volumes_accepted1_array, dims=1)'
tot_volumes_accepted2_array_std = std(tot_volumes_accepted2_array, dims=1)'

volumes_sum = tot_volumes_accepted1_array_mean .+ tot_volumes_accepted2_array_mean
volumes_std = sqrt.( tot_volumes_accepted1_array_std.^2 .+ tot_volumes_accepted2_array_std.^2)

mean_time = mean(ahmi_time_array, dims=1)'
std_time = std(ahmi_time_array, dims=1)'

fig, ax = plt.subplots(1,2, figsize=(13, 4))

ax[1].plot(dim_range, mean_time, color=plt.cm.Oranges(0.8))
ax[1].fill_between(dim_range, mean_time[:,1] .- std_time[:,1], mean_time[:,1] .+ std_time[:,1], alpha=0.4, color=plt.cm.Oranges(0.3),)
ax[1].set_xlabel("# Dimensions")
ax[1].set_ylabel("Integration Time, [sec.]")
# ax[1].set_xlim(2, 31)

ax[2].plot(dim_range, volumes_sum, color=plt.cm.Oranges(0.8))
ax[2].fill_between(dim_range, volumes_sum[:,1] .- volumes_std[:,1], volumes_sum[:,1] .+ volumes_std[:,1], alpha=0.4, color=plt.cm.Oranges(0.3),)
ax[2].set_xlabel("# Dimensions")
ax[2].set_ylabel("# Hyper Rectangles")
# ax[2].set_xlim(2, 31)

# fig.savefig("/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/NormalDistributionData/time_cubes.pdf", bbox_inches = "tight")

In [ ]:
mcmc_mean_time = mean(mcmc_time_array, dims=1)'
mcmc_std_time = std(mcmc_time_array, dims=1)'


plot(dim_range, mcmc_mean_time, ribbon=(mcmc_std_time, mcmc_std_time),
    linecolor=line_colors[1], fillcolor=line_colors[4], fillalpha=0.3,  label="AHMI mean time")

scatter!([dim_array...], [mcmc_time_array...],
    lw=0.0, markeralpha=0.5,
    markercolor=line_colors[3], 
    markerstrokewidth=0.1, label="MCMC time")

plot!(box=true, 
    grid=false, 
    size=(800, 500),
    xticks = dim_range,
#     xlim=(0, 22),
    title = TITLE,
    xlabel = L"N_{dim}", 
    ylabel = "MCMC time, [sec]", 
)

In [ ]:
tot_volumes_accepted1_array_mean = mean(tot_volumes_accepted1_array, dims=1)'
tot_volumes_accepted2_array_mean = mean(tot_volumes_accepted2_array, dims=1)'

tot_volumes_accepted1_array_std = std(tot_volumes_accepted1_array, dims=1)'
tot_volumes_accepted2_array_std = std(tot_volumes_accepted2_array, dims=1)'


plot(dim_range, tot_volumes_accepted1_array_mean, ribbon=(tot_volumes_accepted1_array_std, tot_volumes_accepted1_array_std),
    linecolor=line_colors[1], fillcolor=line_colors[4], fillalpha=0.2,  label="Data Set 1: <N>")

scatter!([dim_array...], [tot_volumes_accepted1_array...],
    lw=0.0, markeralpha=0.5,
    markercolor=line_colors[3], 
    markerstrokewidth=0.1, label="Data Set 1: N_vol")

plot!(dim_range, tot_volumes_accepted2_array_mean, ribbon=(tot_volumes_accepted2_array_std, tot_volumes_accepted2_array_std),
    linecolor=line_colors[13], fillcolor=line_colors[14], fillalpha=0.2, label="Data Set 2: <N>")

scatter!([dim_array...], [tot_volumes_accepted2_array...],
    lw=0.0, markeralpha=0.5,
    markercolor=line_colors[13], 
    markerstrokewidth=0.1,  label="Data Set 2: N_vol")

plot!(box=true, 
    grid=false, 
    size=(800, 500),
    xticks = dim_range,
#     xlim=(0, 22),
    title = TITLE,
    xlabel = L"N_{dim}", 
    ylabel = L"N volumes", 
)

# Comparison iid-MCMC: 

In [ ]:
# PATH_1 = "../../AHMI_publication/NormalDistributionData/normal_dist_3-iid.jld"
# PATH_2 = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/NormalDistributionData/normal_dist_3.jld"
# PATH_2 = "../../AHMI_publication/NormalDistributionData/normal_dist_4-iid.jld" #f_min/f_max

# PATH_2 = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/CaushyDistributionData/mmod-caushy_dist_4-ffcor-2.jld"
# PATH_1 = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/CaushyDistributionData/mmod-caushy_dist_4.jld"

# PATH_1 = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/FunnelDistributionData/funnel_dist_2.jld"
# PATH_2 = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/FunnelDistributionData/funnel_dist_2-ffcor.jld"
# TITLE = "AHMI Tests (Funnel Distribution)"

PATH_1 = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/GaussShellDistributionData/gaus_shell_dist_5.jld"
PATH_2 = "/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/GaussShellDistributionData/gaus_shell_dist_5-ffcor.jld"
TITLE = "AHMI Tests (Funnel Distribution)"

saved_1 = load(PATH_1);
saved_2 = load(PATH_2);

In [ ]:
dim_array = saved_1["dim_array"]
integrals_ahmi = saved_1["integrals_ahmi_array"]
uns_ahmi = saved_1["uns_ahmi_array"]
integrals_true = saved_1["integrals_true_array"]

dim_range_1 = sort(unique(dim_array));
integrals_1 = exp.(integrals_ahmi .- integrals_true)
unsertanity_ahmi_1 = exp.(uns_ahmi .- integrals_true);
unsertanity_scatter_1 = std(integrals_1, dims=1)'

mean_integrals_1 = mean(integrals_1, dims=1)'
unsertanity_ahmi_1 = mean(unsertanity_ahmi_1, dims=1)';

In [ ]:
dim_array = saved_2["dim_array"]
integrals_ahmi = saved_2["integrals_ahmi_array"]
uns_ahmi = saved_2["uns_ahmi_array"]
integrals_true = saved_2["integrals_true_array"]

dim_range_2 = sort(unique(dim_array));
integrals_2 = exp.(integrals_ahmi .- integrals_true)
unsertanity_ahmi_2 = exp.(uns_ahmi .- integrals_true);
unsertanity_scatter_2 = std(integrals_2, dims=1)'

mean_integrals_2 = mean(integrals_2, dims=1)'
unsertanity_ahmi_2 = mean(unsertanity_ahmi_2, dims=1)';

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(13, 6))

ax.axhline(1, c="red", label="Truth")

ax.plot(dim_range_2, mean_integrals_2[:,1], label=L"MCMC: $\langle\hat{I}\rangle$", color=plt.cm.Blues(0.8), alpha=0.5)
ax.fill_between(dim_range_2, mean_integrals_2[:,1] .- unsertanity_scatter_2[:,1], mean_integrals_2[:,1] .+ unsertanity_scatter_2[:,1], alpha=0.5, color=plt.cm.Blues(0.3), label=L"MCMC: $\sigma(\hat{I})$" )
ax.plot(dim_range_2, mean_integrals_2[:,1] .- unsertanity_ahmi_2[:,1], alpha=0.5, ls="--", color=plt.cm.Blues(0.8),  label=L"MCMC: $\langle\sigma\rangle$",)
ax.plot(dim_range_2, mean_integrals_2[:,1] .+ unsertanity_ahmi_2[:,1], alpha=0.5, ls="--", color=plt.cm.Blues(0.8))

ax.plot(dim_range_1, mean_integrals_1[:,1], label=L"i.i.d.: $\langle\hat{I}\rangle$", color=plt.cm.Oranges(0.8), alpha=0.5)
ax.fill_between(dim_range_1, mean_integrals_1[:,1] .- unsertanity_scatter_1[:,1], mean_integrals_1[:,1] .+ unsertanity_scatter_1[:,1], alpha=0.5, color=plt.cm.Oranges(0.3), label=L"i.i.d.: $\sigma(\hat{I})$" )
ax.plot(dim_range_1, mean_integrals_1[:,1] .- unsertanity_ahmi_1[:,1], alpha=0.5, ls="--", color=plt.cm.Oranges(0.8),  label=L"i.i.d.: $\langle\sigma\rangle$",)
ax.plot(dim_range_1, mean_integrals_1[:,1] .+ unsertanity_ahmi_1[:,1], alpha=0.5, ls="--", color=plt.cm.Oranges(0.8))


ax.set_xlabel("# Dimensions")
ax.set_ylabel("Ratio to Truth")

ax.legend(loc="upper left", frameon=true, framealpha=0.8, ncol=1)

# fig.savefig("/home/iwsatlas1/vhafych/MPP-Project/AHMI_publication/NormalDistributionData/normal_benchmarking.pdf", bbox_inches = "tight")